In [17]:
# Used for indices based on the indice list
def dropdown_options(listname):
	dropdown_list = []
	for item in listname:
		dropdown_list.append({'label':str(item), 'value':item})
	return dropdown_list

# Used to convert the symbol of index to the name of the index
def get_index_symbol(index_list):
    new_list = []
    for i in range(len(index_list)):
        new_list.append(list(indices[indices['Name'] == index_list[i]]['Symbol'])[0])
    return new_list

# Convert dataset from json back to pandas
# Function should be used for dcc.store as all dcc.store variables must be converted to json
def data_to_pandas(dataset):
    json_dataframe = json.loads(dataset)
    json_dataframe = pd.read_json(json.dumps(json_dataframe))
    return json_dataframe

# Prepares the dataset for the correlation graph
# Function should be used to take in dcc.store variable and covert that from json and create the df
def correlation_data(func_data, ticker_input_func, radio_input):
    func_data = data_to_pandas(func_data)
    column_list = []
    for i in range(len(ticker_input_func)):
        column_list.append(ticker_input_func[i])
    df = pd.DataFrame(columns = column_list)
    for ticker in ticker_input_func:
        df[ticker] = func_data[str((radio_input,ticker))]
    df = df.corr()
    return df

# Obtain stock analytics dataset and put it into a dictionary
def obtain_stock_analytics_data(ticker_input):
    ticker_input = ticker_input.split(",")    
    stock_analytics_data = []
    for ticker in ticker_input:
        url = 'https://www.marketwatch.com/investing/stock/'+ticker.lower() + '/company-profile?mod=mw_quote_tab'
        time.sleep(2.5)
        stock_analytics_data.append(pd.read_html(url))
    return stock_analytics_data
        
# convert the stock analytics into a readable table which is then used for graph data
def stock_analytics(summary_data, ticker_input, index):
        # Df 4 for valuation
        # Df 6 for liquidity
        # Df 7 for profitability
        # Df 8 for capitalization
        # Df 10 for insider actions
    ticker_input = ticker_input.split(",")  
    summary_data_dict = {}
    summary_data_dict['Description'] = summary_data[0][index][0]
    for i in range(len(ticker_input)):
        summary_data_dict[ticker_input[i]] = summary_data[i][index][1]
    summary_data_df = pd.DataFrame(summary_data_dict)
    return summary_data_df

# Obtain necessary news articles based on ticker_input, start date and end date
def get_news_articles(ticker_input_func,start_date_func,end_date_func):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    
    ticker_input_func = ticker_input_func.split(',')
    df_news = pd.DataFrame()     
    
    for ticker in ticker_input_func:
        temp_news = pd.DataFrame()
        googlenews=GoogleNews(start= start_date_func,end=end_date_func)
        googlenews.search(yf.Ticker(ticker).info['longName'])
        temp_news = pd.DataFrame(googlenews.result())
        temp_news['Ticker'] = ticker
        df_news = df_news.append(temp_news)
    return df_news

# Cleanup news articles, obtain article title, media, datetime and full text
def news_articles_cleanup(df):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    news_list = []
    df = df.reset_index()
    for ind in df.index:
        try:
            dict_news={}
            article = Article(df['link'][ind],config=config)
            article.download()
            article.parse()
            article.nlp()
            dict_news['Date']=df['date'][ind]
            dict_news['Ticker']=df['Ticker'][ind]
            dict_news['Media']=df['media'][ind]
            dict_news['Title']=article.title
            dict_news['Article']=article.text
            news_list.append(dict_news)
        except:
            pass
    cleaned_df = pd.DataFrame(news_list)
    cleaned_df.replace('',np.nan, inplace=True)
    cleaned_df = cleaned_df.dropna()
    return cleaned_df

# Sentiment analysis of scores based on news_articles_cleanup dataset
def sentiment_scores_df(dataset):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    sentiment_list = []
    sentiment_dict = {}
    sid = SentimentIntensityAnalyzer()
    try: 
        dataset = dataset.reset_index()
    except:
        pass
    # Creates the sentiment dictionary which will then be passed off as a json file
    for i in range(len(dataset['Article'])):
        sentiment_dict = {}
        polarity_scores = sid.polarity_scores(dataset['Article'][i])
        sentiment_dict['Ticker'] = dataset['Ticker'][i]
        #sentiment_dict['Article'] = dataset['Article'][i]
        sentiment_dict['Date'] = dataset['Date'][i]
        sentiment_dict['Negative'] = polarity_scores['neg']
        sentiment_dict['Positive'] = polarity_scores['pos']
        sentiment_dict['Neutral'] = polarity_scores['neu']
        sentiment_dict['Compound'] = polarity_scores['compound']
        sentiment_list.append(sentiment_dict)
    return pd.DataFrame(sentiment_list)

# Use the dictionary for the title
radio_dict = {'Adj Close':'Adjusted Close', 'Open':'Open','Close':'Close','High':'High','Low':'Low','Volume':'Volume'}

# Use the dictionary for the stock analytics
stock_analytics_dict = {4:'Valuation',6:'Liquidity',7:'Profitability',8:'Capitalization'}

In [19]:
# Import necessary libraries
# Impot dash related libraries
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
# Visualization libraries
import plotly.graph_objs as go
import seaborn as sns
import plotly.express as px
# Import pandas related libraries
import pandas as pd
import pandas_datareader.data as web
import numpy as np
# Import news articles libraries
from GoogleNews import GoogleNews
from newspaper import Article
from newspaper import Config
import yfinance as yf
# Import sentiment analyzer libraries
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Import other libraries
import datetime as dt
import json
import time

# External stylesheets for the application
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Setup dash application
app = dash.Dash(external_stylesheets=external_stylesheets)

# Lists used for DCC dropdowns
indices = pd.read_csv('List_of_Indices.csv')

# color used for H1 (Title)
color = {'background':'#111111','text':'#000709'}

# App layout
app.layout = html.Div(children = [
    # H1 Title for the Dashboard
    html.H1('Portfolio Dashboard', style = {'textAlign':'center', 'color':color['text'], 'size':20, 'font-weight':'bold'}),
    
# ------------------------------------------ PART 1 - INDEX UI ---------------------------------------
    
    # H2 Title for the dashboard
    html.H2('Index Analysis', style = {'textAlign':'left', 'color':color['text'], 'size':14, 'font-weight':'bold'}),
    # Div used to have 2 labels
    html.Div(className = 'row', style = {'display':'flex'}, children = [
        html.Label('Please select the indices:', style = {'textAlign':'left', 'font-weight':'bold'}),
        html.Label('Please select the appropriate dates:', style = {'margin':'auto', 'font-weight':'bold'})      
    ]),
    html.Div(className = 'row', style = {'display':'flex'},children = [
        dcc.Dropdown(id = 'indice_selector', options = dropdown_options(list(indices['Name'])), value = indices['Name'][0:2],
                multi = True, style = {'font-size': '14px','width':'400px'}),
        dcc.DatePickerRange(id = 'indice_date_selector',start_date_placeholder_text="Start Period", end_date_placeholder_text="End Period",
                    style={'font-size': '14px','width':'400px', 'margin':'auto'})
    ]),
    html.Label('Please select the type of graph:', style = {'textAlign':'left', 'font-weight':'bold'}),
    # Radio Items used to update the index graph
    dcc.RadioItems(id = 'index_radio_items',options = [
                                            {'label':'Adjusted Close', 'value':'Adj Close'},
                                            {'label':'Open','value':'Open'},
                                            {'label':'Close','value':'Close'},
                                            {'label':'High','value':'High'},
                                            {'label':'Low','value':'Low'},
                                            {'label':'Volume','value':'Volume'}],
                   value = 'Volume',
                   labelStyle={'display': 'inline-block'}),
    # Index Graph - shows line grpah based on inputs from user
    dcc.Graph(id = 'indice_graph'),
    # Correlation Graph shows correlation between indexs based on inputs from user
    dcc.Graph(id = 'indice_correlation'),
    # Store value for the indice graph and indice correlation plot
    dcc.Store(id='indice_graph_data'),

# ------------------------------------------ PART 2 - STOCK Analysis UI ---------------------------------------
    
    # Stock portion of the application
    html.H2('Stock Analysis', style = {'textAlign':'left', 'color':color['text'], 'font-weight':'bold'}),
    # Div used to format the labels for ticker and dates
    html.Div(className = 'row', style = {'display':'flex'}, children = [
        html.Label('Please input the tickers and click submit:', style = {'textAlign':'left', 'font-weight': 'bold'}),
        html.Label('Please select the appropriate dates:', style = {'margin':'auto', 'font-weight': 'bold'})
    ]),
    # Div used to format the inputs for stock graph and stock correlation graph
    html.Div(className = 'row', style = {'display':'flex'}, children = [
        dcc.Input(id = 'ticker_input', type = 'text', style = {'width':'500px', 'font-size':'14px'}),
        dcc.DatePickerRange(id = 'stock_date_selector',start_date_placeholder_text="Start Period", end_date_placeholder_text="End Period",
            style={'font-size': '14px','width':'400px', 'margin':'auto'})
    ]),
     html.Label('Please select the type of graph:', style = {'textAlign':'left', 'font-weight': 'bold'}),
    # Radio Items used to update the stock graph
    dcc.RadioItems(id = 'stock_radio_items',options = [
                                            {'label':'Adjusted Close', 'value':'Adj Close'},
                                            {'label':'Open','value':'Open'},
                                            {'label':'Close','value':'Close'},
                                            {'label':'High','value':'High'},
                                            {'label':'Low','value':'Low'},
                                            {'label':'Volume','value':'Volume'}],
                   value = 'Volume',
                   labelStyle={'display': 'inline-block'}),
    dcc.Store(id = 'stock_graph_data'),
    # Submit button used to update the stock graph
    html.Button('Submit', id = 'stock_button', n_clicks = 0),
    # Stock graph - shows lien graph based on inputs from the user
    dcc.Graph(id = 'stock_graph'),
    # Correlation graph - shows correlation based on inputs from the user
    dcc.Graph(id = 'stock_correlation'),
    
# ------------------------------------------ PART 3 - STOCK Analytics UI ---------------------------------------
    
    dcc.Store(id = 'stock_analytics_data'),
    html.H2('Stock Analytics', style = {'textAlign':'left', 'color':color['text'], 'font-weight':'bold'}),
    html.Div(className = 'row', style = {'display':'flex'}, children = [
        html.Label('Stock Valuation (MarketWatch)', style = {'textAlign':'left', 'font-weight': 'bold'}),
        html.Label('Stock Liquidity (MarketWatch)', style = {'margin':'auto', 'font-weight': 'bold'})
    ]),
    html.Div(className = 'row', style = {'display':'flex'}, children = [
        dcc.Graph(id = 'stock_valuation', style = {'width':'900px', 'height':'680px'}),
        dcc.Graph(id = 'stock_liquidity', style={'width':'900px', 'height':'680px', 'margin':'auto'})
    ]),
    html.Div(className = 'row', style = {'display':'flex'}, children = [
        html.Label('Stock Profitability (MarketWatch)', style = {'textAlign':'left', 'font-weight': 'bold'}),
        html.Label('Stock Capitalization (MarketWatch)', style = {'margin':'auto', 'font-weight': 'bold'})
    ]),
    html.Div(className = 'row', style = {'display':'flex'}, children = [
        dcc.Graph(id = 'stock_profitability', style = {'width':'900px', 'height':'680px'}),
        dcc.Graph(id = 'stock_capitalization', style={'width':'900px', 'height':'680px', 'margin':'auto'})
    ]),
    
# ------------------------------------------ PART 4 - ARTICLES UI ---------------------------------------
    
    dcc.Store(id = 'articles_data'),
    html.H2('Stock Sentiment Analysis', style = {'textAlign':'left','color':color['text'], 'font-weight':'bold'}),
    html.Label('Please select the appropriate sentiment graph: ', style = {'textAlign':'left', 'font-weight': 'bold'}),
    dcc.RadioItems(id = 'sentiment_radio_items',options = [
                                                {'label':'Positive', 'value':'Positive'},
                                                {'label':'Negative','value':'Negative'},
                                                {'label':'Neutral','value':'Neutral'},
                                                {'label':'Compound','value':'Compound'}],
                  value = 'Compound', labelStyle={'display': 'inline-block'}                  
                  ),
    html.Div(className = 'row', style = {'display':'flex'}, children = [
        dcc.Graph(id = 'line_graph_sentiment_graph', style = {'width':'900px', 'height':'680px'}),
        dcc.Graph(id = 'bar_graph_sentiment_graph', style={'width':'900px', 'height':'680px', 'margin':'auto'})
    ])

# ------------------------------------------ PART 5 - ARTICLES LOOKUP ---------------------------------------
])


# ------------------------------------------ PART 1 - INDEX BACKEND ---------------------------------------

# DCC Input that downloads the index data and keeps it for use by graph
@app.callback(Output('indice_graph_data', 'data'), [Input('indice_selector', 'value'),
                                                   Input('indice_date_selector','start_date'),
                                                   Input('indice_date_selector','end_date')])
def update_indice_data(indice_name, indice_start_date, indice_end_date):
    indice_name = get_index_symbol(indice_name)
    data = web.DataReader(indice_name,'yahoo', indice_start_date, indice_end_date).reset_index()
    return data.to_json(date_format = 'iso', orient = 'records')

# Index Graph showing multiple indexes based on user input from indice selector (id = 'indice_selector') and date picker (id = 'indice_date_selector')
@app.callback(Output('indice_graph', 'figure'), [Input('indice_selector','value'), 
                                                 Input('indice_date_selector','start_date'),
                                                 Input('indice_date_selector','end_date'),
                                                 Input('index_radio_items', 'value'),
                                                 Input('indice_graph_data','data')])
def update_index_line_graph(indice_name, indice_start_date, indice_end_date, radio_item, json_data):
    graph_data = []
    full_index_name = indice_name
    indice_name = get_index_symbol(indice_name)
    data = data_to_pandas(json_data)
    for index in indice_name:
        graph_data.append(go.Scatter(x = data[str(('Date',''))], y = data[str((radio_item,index))], mode = 'lines', name = full_index_name[indice_name.index(index)]))
    graph_layout = go.Layout(title = '{} Line Chart for {}'.format(radio_dict.get(radio_item),",".join(full_index_name)), xaxis_title = 'Date', yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }

# Correlation Graph based on user input from indice selector (id = 'indice_selector') and date picker (id = 'indice_date_selector')
@app.callback(Output('indice_correlation', 'figure'), [Input('indice_selector', 'value'), 
                                                       Input('indice_date_selector','start_date'),
                                                       Input('indice_date_selector','end_date'),
                                                       Input('index_radio_items', 'value'),
                                                       Input('indice_graph_data','data')])
def update_indice_corr_figure(indice_name, indice_start_date, indice_end_date, radio_item, json_data):
    full_index_name = indice_name
    indice_name = get_index_symbol(indice_name)
    corr_data = correlation_data(json_data, indice_name, radio_item)
    # data = web.DataReader(indice_name,'yahoo', indice_start_date, indice_end_date).reset_index()
    fig = px.imshow(corr_data,labels = dict(x = 'Index', y = 'Index', color = 'Correlation'), x = full_index_name, y = full_index_name, title = 'Correlation Graph of {} for {}'.format(
                                                                                                                                                        radio_dict.get(radio_item),",".join(full_index_name)))
    fig.update_layout(title_x=0.5)
    return fig

# ------------------------------------------ PART 2 - STOCK BACKEND ---------------------------------------

# DCC Input that downloads the stock data and keeps it for use for graphs
@app.callback(Output('stock_graph_data', 'data'), [Input('stock_button', 'n_clicks'),
                                                  Input('stock_date_selector','start_date'),
                                                  Input('stock_date_selector','end_date')],
                                                [State('ticker_input','value')]) # The state is so that the graph does not update until user clicks on submit button
def update_stock_data(n_clicks, stock_start_date, stock_end_date, ticker_input):
    ticker_input = ticker_input.split(",")
    data = web.DataReader(ticker_input,'yahoo', stock_start_date, stock_end_date).reset_index()
    return data.to_json(date_format = 'iso', orient = 'records')

# Stock Graph showing multiple stocks based on user input from stock selector (id = 'stock_selector') and date picker (id = 'stock_date_selector')
@app.callback(Output('stock_graph', 'figure'), [Input('stock_button','n_clicks'), 
                                                 Input('stock_date_selector','start_date'),
                                                 Input('stock_date_selector','end_date'),
                                                 Input('stock_radio_items', 'value'),
                                                 Input('stock_graph_data','data'),
                                                 Input('ticker_input','value')])
def update_stock_line_graph(n_clicks, stock_start_date, stock_end_date, radio_item, json_data, ticker_input):
    graph_data = []
    ticker_input = ticker_input.split(",")
    data = data_to_pandas(json_data)
    for ticker in ticker_input:
        graph_data.append(go.Scatter(x =  data[str(('Date',''))], y = data[str((radio_item,ticker))], mode = 'lines', name = ticker))
    graph_layout = go.Layout(title = '{} Line Chart for {}'.format(radio_dict.get(radio_item),",".join(ticker_input)), xaxis_title = 'Date', yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }

# Correlation Graph based on user input form ticker selector (id = 'ticker_input'), date picker (id = 'stock_date_selector') and update button (id = 'stock_button')
@app.callback(Output('stock_correlation', 'figure'), [Input('ticker_input', 'value'), 
                                                       Input('stock_date_selector','start_date'),
                                                       Input('stock_date_selector','end_date'),
                                                       Input('stock_radio_items', 'value'),
                                                       Input('stock_graph_data','data')])
def update_stock_corr_figure(stock_input, stock_start_date, stock_end_date, radio_item, json_data):
    stock_input = stock_input.split(",")
    corr_data = correlation_data(json_data, stock_input, radio_item)
    fig = px.imshow(corr_data,labels = dict(x = 'Ticker', y = 'Ticker', color = 'Correlation'), x = stock_input, y = stock_input, title = 'Correlation Graph of {} for {}'.format(radio_dict.get(radio_item),stock_input))
    fig.update_layout(title_x=0.5)
    return fig
    
# ------------------------------------------ PART 3 - STOCK ANALYTICS BACKEND ---------------------------------------

# DCC Input that downloads the stock analytics data and keeps it for use for the 4 graphs
@app.callback(Output('stock_analytics_data', 'data'), [Input('stock_button', 'n_clicks')],
                                                      [State('ticker_input','value')]) # The state is so that the graph does not update until user clicks on submit button
def update_stock_data(n_clicks, ticker_input):
    # Get the stock analytics from marketwatch
    stock_analytics_data = obtain_stock_analytics_data(ticker_input)
    # Create empty dataframe to obtain the stock analytics for user
    df = pd.DataFrame()
    for i in [4,6,7,8]:
        if i ==4:
            temp_df = stock_analytics(stock_analytics_data,ticker_input,i)
            temp_df['Category'] = stock_analytics_dict.get(i)
            df = temp_df.copy()
        else:
            temp_df = stock_analytics(stock_analytics_data, ticker_input, i)
            temp_df['Category'] = stock_analytics_dict.get(i)
            df = df.append(temp_df, ignore_index = True)    
    return df.to_json(date_format = 'iso', orient = 'records')

# Create graph for Stock analytics for Valuation Data
@app.callback(Output('stock_valuation', 'figure'), [Input('ticker_input','value'),
                                                    Input('stock_analytics_data','data')])
def update_index_line_graph(ticker_input, json_data):
    ticker_input = ticker_input.split(",")
    data = data_to_pandas(json_data)
    # Take the data for Valuation category only
    data = data[data['Category'] == 'Valuation']
    graph_data = []
    for ticker in ticker_input:
        graph_data.append(go.Bar(x =  data['Description'], y = data[ticker], name = ticker))
    graph_layout = go.Layout(title = 'Stock Valuation Bar Chart for {}'.format(",".join(ticker_input)), yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }

# Create graph for Stock analytics for Liquidity Data
@app.callback(Output('stock_liquidity', 'figure'), [Input('ticker_input','value'),
                                                    Input('stock_analytics_data','data')])
def update_index_line_graph(ticker_input, json_data):
    ticker_input = ticker_input.split(",")
    data = data_to_pandas(json_data)
    # Take the data for Valuation category only
    data = data[data['Category'] == 'Liquidity']
    graph_data = []
    for ticker in ticker_input:
        graph_data.append(go.Bar(x =  data['Description'], y = data[ticker], name = ticker))
    graph_layout = go.Layout(title = 'Stock Liquidity Bar Chart for {}'.format(",".join(ticker_input)), yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }

# Create graph for Stock analytics for Profitability Data
@app.callback(Output('stock_profitability', 'figure'), [Input('ticker_input','value'),
                                                    Input('stock_analytics_data','data')])
def update_index_line_graph(ticker_input, json_data):
    ticker_input = ticker_input.split(",")
    data = data_to_pandas(json_data)
    # Take the data for Valuation category only
    data = data[data['Category'] == 'Profitability']
    graph_data = []
    for ticker in ticker_input:
        graph_data.append(go.Bar(x =  data['Description'], y = data[ticker], name = ticker))
    graph_layout = go.Layout(title = 'Stock Profitability Bar Chart for {}'.format(",".join(ticker_input)), yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }

# Create graph for Stock analytics for Capitalization Data
@app.callback(Output('stock_capitalization', 'figure'), [Input('ticker_input','value'),
                                                        Input('stock_analytics_data','data')])
def update_index_line_graph(ticker_input, json_data):
    ticker_input = ticker_input.split(",")
    data = data_to_pandas(json_data)
    # Take the data for Valuation category only
    data = data[data['Category'] == 'Capitalization']
    graph_data = []
    for ticker in ticker_input:
        graph_data.append(go.Bar(x =  data['Description'], y = data[ticker], name = ticker))
    graph_layout = go.Layout(title = 'Stock Capitalization Bar Chart for {}'.format(",".join(ticker_input)), yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }
    
# ------------------------------------------ PART 4 - ARTICLES BACKEND ---------------------------------------

# DCC Input that downloads the stock news data for use from sentiment analyzer    
@app.callback(Output('articles_data', 'data'), [Input('stock_button', 'n_clicks'),
                                                Input('stock_date_selector', 'start_date'),
                                                Input('stock_date_selector','end_date')],
                                                [State('ticker_input','value')])
def update_sentiment_data(n_clicks,ticker_start_date, ticker_end_date, ticker_input):
    df1 = get_news_articles(ticker_input, ticker_start_date, ticker_end_date)
    df2 = news_articles_cleanup(df1)
    df3 = sentiment_scores_df(df2)
    return df3.to_json(date_format = 'iso', orient = 'records')

# Line Graph Input that visualizes the sentiment analysis data based on 
@app.callback(Output('line_graph_sentiment_graph', 'figure'), [Input('articles_data','data'),
                                                               Input('sentiment_radio_items', 'value')])
def update_sentiment_line_graph(json_data, sentiment_radio_item):
    data = data_to_pandas(json_data)
    ticker_input = list(data['Ticker'].unique())
    data.sort_values('Date').reset_index(drop = True)
    graph_data = []
    for ticker in ticker_input:
            graph_data.append(go.Scatter(x =  data['Date'], y = data[data['Ticker'] == ticker][sentiment_radio_item], mode = 'markers', name = ticker))
    graph_layout = go.Layout(title = '{} Sentiment Scatter Chart for {}'.format(sentiment_radio_item,",".join(ticker_input)), xaxis_title = 'Date', yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }

# Bar Graph Input that visualizes the sentiment analysis data based on ticker
@app.callback(Output('bar_graph_sentiment_graph', 'figure'), [Input('articles_data','data')])
def update_sentiment_line_graph(json_data):
    data = data_to_pandas(json_data)
    ticker_input = list(data['Ticker'].unique())
    graph_data = []
    for ticker in ticker_input:
        mydict = {}
        mydict['Ticker'] = ticker
        mydict['Negative'] = data[data['Ticker'] == ticker]['Negative'].mean()
        mydict['Positive'] = data[data['Ticker'] == ticker]['Positive'].mean()
        mydict['Neutral'] = data[data['Ticker'] == ticker]['Neutral'].mean()
        mydict['Compound'] = data[data['Ticker'] == ticker]['Compound'].mean()
        graph_data.append(mydict)
    
    data = pd.DataFrame(graph_data)
    columns_list = list(data.columns[1:])
    
    for ticker in ticker_input:
        for j in range(len(columns_list)):
            graph_data.append(go.Bar(x =  data[data['Ticker'] == ticker]['Ticker'], y = data[data['Ticker'] == ticker][columns_list[j]], name = columns_list[j]))
    graph_layout = go.Layout(title = 'Sentiment Bar Chart for {}'.format(",".join(ticker_input)), xaxis_title = 'Ticker', yaxis_title = "Value") 
    
    return{
        'data':graph_data,
        'layout':graph_layout
    }


if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Dec/2021 20:41:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:18] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:18] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:18] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:18] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Spyder\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Spyder\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Spyder\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "D:\Spyder\lib\site-packages\dash\_callback.py", line 

127.0.0.1 - - [23/Dec/2021 20:41:18] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Spyder\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Spyder\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Spyder\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "D:\Spyder\lib\site-packages\dash\_callback.py", line 

...

127.0.0.1 - - [23/Dec/2021 20:41:37] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Spyder\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Spyder\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Spyder\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "D:\Spyder\lib\site-packages\dash\_callback.py", line 

127.0.0.1 - - [23/Dec/2021 20:41:39] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Spyder\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Spyder\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Spyder\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "D:\Spyder\lib\site-packages\dash\_callback.py", line 

127.0.0.1 - - [23/Dec/2021 20:41:39] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [23/Dec/2021 20:41:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:41:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:42:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:42:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 20:42:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.

In [16]:
df1 = get_news_articles('MSFT,FB','01/01/2021','01/12/2021')
df2 = news_articles_cleanup(df1)

D:\Spyder\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning:

The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html

D:\Spyder\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning:

The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html



In [14]:
df2

,Date,Ticker,Media,Title,Article
0,2 Jan 2021,MSFT,Microsoft News,Re-imagining manufacturing: Putting digital te...,Re-imagining manufacturing: Putting digital te...
1,2 Jan 2021,MSFT,Threatpost,Mimecast Certificate Hacked in Microsoft Email...,A Mimecast-issued certificate used to authenti...
2,2 Jan 2021,MSFT,Microsoft Blog,Microsoft launches 2021 Council for Digital Good,"Starting today, Microsoft is inviting applicat..."
3,2 Jan 2021,MSFT,Business Wire,SymphonyCare WellStack Now Available in the Mi...,"MADISON, Wis.--(BUSINESS WIRE)--SymphonyCare, ..."
4,2 Jan 2021,MSFT,Microsoft News,Introducing Surface Pro 7+ for Business and Su...,Surface Pro 7+ joins the Surface for Business ...
5,2 Jan 2021,MSFT,Neowin,Brad Anderson is leaving Microsoft and joining...,"Brad Anderson, Corporate Vice President for Co..."
6,2 Jan 2021,MSFT,Windows Central,Microsoft corporate vice president Brad Anders...,"Brad Anderson, Microsoft corporate vice presid..."
7,2 Jan 2021,MSFT,CRN,Hackers Compromise Mimecast Certificate For Mi...,A sophisticated threat actor compromised a Mim...
8,2 Jan 2021,MSFT,ZDNet,Mimecast says hackers abused one of its certif...,"Image: Mimecast, Romain Morel\n\nMimecast, a c..."
9,2 Jan 2021,MSFT,OnMSFT.com,Microsoft VP Brad Anderson is leaving the comp...,Microsoft corporate vice president of Commerci...


In [6]:
df2_json = df2.to_json(date_format = 'iso', orient = 'records')

In [ ]:
# Obtains newspaper and get articles, titles and summary of articles (DONE)
# Get 10 most recent articles (DONE)
# Put all these 10 recent articles for all tickers into a pandas dataframe (DONE)
# Put all these into dcc.Store
# First DCC Dropdown menu on the left to obtain ticker
# Second DCC Dropdown menu on the right to get the title of article
# Below that should be the summary of the article
# Below that should be news articles by news source for all tickers (count plot)
# Below that should be sentiment analyzer
# sentiment analyzer should show sentiment of title for 10 recent articles for all tickers
# sentiment analyzer should show sentiment of article summary for 10 recent articles for all tickers

from GoogleNews import GoogleNews
from newspaper import Article
from newspaper import Config
import pandas as pd
#config will allow us to access the specified url for which we are #not authorized. Sometimes we may get 403 client error while parsing #the link to download the article.

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
googlenews=GoogleNews(start='12/01/2021',end='12/20/2021')
googlenews.search('MSFT')
result=googlenews.result()
df=pd.DataFrame(result)

list=[]
for ind in df.index:
    try:
        dict={}
        article = Article(df['link'][ind],config=config)
        article.download()
        article.parse()
        article.nlp()
        dict['Date']=df['date'][ind]
        dict['Media']=df['media'][ind]
        dict['Title']=article.title
        dict['Article']=article.text
        list.append(dict)
    except:
        pass
ews_df=pd.DataFrame(list)

In [ ]:
df4 = df4.reset_index()

In [ ]:
df4.sort_values('Date').reset_index(drop = True)

In [ ]:
def get_news_articles(ticker_input_func,start_date_func,end_date_func):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    
    ticker_input_func = ticker_input_func.split(',')
    df_news = pd.DataFrame()     
    
    for ticker in ticker_input_func:
        temp_news = pd.DataFrame()
        googlenews=GoogleNews(start= start_date_func,end=end_date_func)
        googlenews.search(yf.Ticker(ticker).info['longName'])
        temp_news = pd.DataFrame(googlenews.result())
        temp_news['Ticker'] = ticker
        df_news = df_news.append(temp_news)
    return df_news